# О задаче

Решение задачи D чемпионата RuCode осени 2023 направления ИИ моей команды (Мой гитхаб: https://github.com/Lex-TS, сокомандника: https://github.com/Lebedkina)

Для решения задачи использовалась модель DecisionTreeClassifier, перед обучением данные были предобработаны, слова натурального языка векторизованы и представлены в виде наборы чисел. В конце, результаты прогнозирования модели на тестовой выборке записываются в csv файл.

По этой задаче мы получили 76.16 баллов из 100 возможных на открытом датасете, на закрытом 76.2 из 100. По закрытому датасету для задачи D заняли 5 место из 7, 1 место по этой задаче в дивизионе Начинающие заняла команда с 97.79 баллами

В этой задаче нужно было преобразовать слова в понятный компьютеру вид, векторизировать их. Однако учитывая то, что обычные методы векторизации в NLP учитывает контекст слов, их повторяемость в предложениях, абзацах и пр., а у нас даны лишь одиночные слова с ударениями, мы не могли найти подходящий метод для векторизации. Нам не хватило опыта и знаний для применения оптимального алгоритма векторизации, однако мы решили эту проблему по своему.

Мы решили векторизировать буквы в слове, сохраняя их порядок. И, используя дерево решений, решать задачу многоклассовой классификации. А после предсказаний интерпретировать результаты, переведя найденные классы (позиции ударений в слове) в слова с ударениями.

В этом ноутбуке представлено решение, которое мы успели сделать до конца проведения соревнования, оно не улучшено для более точной работы модели, и здесь пока что не исправлены возможные ошибки с максимальной длиной слов и нереалистичной расстановкой ударений в некоторых словах.

## Дополнительный комментарий

Мы пробовали векторизировать слова при помощи: Word2Vec, fastText, мешком слов, HachVectorizer, one-hot, MultiLabelBinarizer, LabelBinarizer. Однако, эти способы показывали плохой результат метрики. Лучше всего показал себя тот вариант, который мы использовали в итоговом решении: закодировать каждую букву в слове, представив слово как вектор чисел, сохраняя положение каждой буквы.

Для классификации пробовали применять модели: XGBClassifier, Gaussian NaiveBayes, алгоритм поиска ближайших соседей, логическую регрессию. Но остановились на дереве решений, так как эта модель обучалась крайне быстро и выдавала лучший результат метрики, по сравнению с другими моделями.

# Загрузка библиотек и данных

In [5]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import tokenize

In [ ]:
# Загрузка данных из своего гугл диска
# from google.colab import drive
# drive.mount('/content/gdrive/')
# data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_D/train_stresses_labels.txt', header=None, names=['Ударения'])

Mounted at /content/gdrive/


In [2]:
#Если загружать данные локально:
data = pd.read_csv('train_stresses_labels.txt', header=None, names=['Ударения'])

In [ ]:
# Загрузка данных из своего гугл диска
# test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_D/public_test_stresses.txt', header=None, names=['Слова'])

In [3]:
test = pd.read_csv('public_test_stresses.txt', header=None, names=['Слова'])

In [4]:
rand_state = 12

##Готовим набор данных

In [6]:
def preprocess(text):
  pattern = '.'
  tok = tokenize.RegexpTokenizer(pattern)
  return tok.tokenize(text)

In [7]:
data['Слова'] = data.apply(lambda row: row['Ударения'].replace('^', ""), axis=1)

In [8]:
data

,Ударения,Слова
0,аа^к,аак
1,аа^ка,аака
2,аа^ке,ааке
3,аа^ки,ааки
4,аа^ков,ааков
...,...,...
588485,ёфика^торе,ёфикаторе
588486,ёфика^ции,ёфикации
588487,ёфика^цию,ёфикацию
588488,ёфика^ция,ёфикация


In [9]:
data['Токены ударений'] = data.apply(lambda row: preprocess(row['Ударения']), axis=1)

In [10]:
data

,Ударения,Слова,Токены ударений
0,аа^к,аак,"[а, а, ^, к]"
1,аа^ка,аака,"[а, а, ^, к, а]"
2,аа^ке,ааке,"[а, а, ^, к, е]"
3,аа^ки,ааки,"[а, а, ^, к, и]"
4,аа^ков,ааков,"[а, а, ^, к, о, в]"
...,...,...,...
588485,ёфика^торе,ёфикаторе,"[ё, ф, и, к, а, ^, т, о, р, е]"
588486,ёфика^ции,ёфикации,"[ё, ф, и, к, а, ^, ц, и, и]"
588487,ёфика^цию,ёфикацию,"[ё, ф, и, к, а, ^, ц, и, ю]"
588488,ёфика^ция,ёфикация,"[ё, ф, и, к, а, ^, ц, и, я]"


In [11]:
data['Токены слов'] = data.apply(lambda row: preprocess(row['Слова']), axis=1)

In [12]:
data

,Ударения,Слова,Токены ударений,Токены слов
0,аа^к,аак,"[а, а, ^, к]","[а, а, к]"
1,аа^ка,аака,"[а, а, ^, к, а]","[а, а, к, а]"
2,аа^ке,ааке,"[а, а, ^, к, е]","[а, а, к, е]"
3,аа^ки,ааки,"[а, а, ^, к, и]","[а, а, к, и]"
4,аа^ков,ааков,"[а, а, ^, к, о, в]","[а, а, к, о, в]"
...,...,...,...,...
588485,ёфика^торе,ёфикаторе,"[ё, ф, и, к, а, ^, т, о, р, е]","[ё, ф, и, к, а, т, о, р, е]"
588486,ёфика^ции,ёфикации,"[ё, ф, и, к, а, ^, ц, и, и]","[ё, ф, и, к, а, ц, и, и]"
588487,ёфика^цию,ёфикацию,"[ё, ф, и, к, а, ^, ц, и, ю]","[ё, ф, и, к, а, ц, и, ю]"
588488,ёфика^ция,ёфикация,"[ё, ф, и, к, а, ^, ц, и, я]","[ё, ф, и, к, а, ц, и, я]"


## Определяем частоту букв

In [13]:
from collections import Counter

Тестируем работу Counter

In [14]:
test_counter = Counter(data['Токены ударений'][0])

In [15]:
test_counter

Counter({'а': 2, '^': 1, 'к': 1})

In [16]:
test_counter.most_common(10)

[('а', 2), ('^', 1), ('к', 1)]

Считаем частоту букв во всех словах

In [17]:
chars = Counter()

In [18]:
for txt in data['Токены ударений']:
    chars.update(txt)

Количество символов во всех словах

In [19]:
len(chars)

34

In [20]:
chars

Counter({'а': 568791,
         '^': 588490,
         'к': 200543,
         'е': 492524,
         'и': 533689,
         'о': 583396,
         'в': 395981,
         'м': 218550,
         'у': 176247,
         'н': 368859,
         'г': 89993,
         'ч': 79081,
         'р': 363020,
         'ц': 35524,
         'с': 356297,
         'й': 101042,
         'ы': 121547,
         'щ': 58328,
         'х': 54346,
         'б': 86853,
         'д': 125651,
         'ю': 92289,
         'ж': 39653,
         'з': 114509,
         'я': 169692,
         'т': 316268,
         'л': 242565,
         'ь': 98192,
         'э': 13891,
         'ш': 135489,
         'п': 214679,
         'ф': 26172,
         'ъ': 2088,
         'ё': 373})

Самые популярные буквы

In [21]:
chars.most_common(10)

[('^', 588490),
 ('о', 583396),
 ('а', 568791),
 ('и', 533689),
 ('е', 492524),
 ('в', 395981),
 ('н', 368859),
 ('р', 363020),
 ('с', 356297),
 ('т', 316268)]

## Создаем словарь, упорядоченный по частоте

В словаре будем использовать 2 специальных кода:
- Код заполнитель: 0
- Неизвестное слово: 1

Нумерация слов в словаре начинается с 2.

In [22]:
# Словарь, отображающий слова в коды
char_to_index = dict()
# Словарь, отображающий коды в слова
index_to_char = dict()

Максимальное количество обрабатываемых символов

In [23]:
max_chars = len(chars) + 2 #36

Создаем словари

In [24]:
for i, char in enumerate(chars.most_common(max_chars - 2)):
    char_to_index[char[0]] = i + 2
    index_to_char[i + 2] = char[0]

In [25]:
char_to_index

{'^': 2,
 'о': 3,
 'а': 4,
 'и': 5,
 'е': 6,
 'в': 7,
 'н': 8,
 'р': 9,
 'с': 10,
 'т': 11,
 'л': 12,
 'м': 13,
 'п': 14,
 'к': 15,
 'у': 16,
 'я': 17,
 'ш': 18,
 'д': 19,
 'ы': 20,
 'з': 21,
 'й': 22,
 'ь': 23,
 'ю': 24,
 'г': 25,
 'б': 26,
 'ч': 27,
 'щ': 28,
 'х': 29,
 'ж': 30,
 'ц': 31,
 'ф': 32,
 'э': 33,
 'ъ': 34,
 'ё': 35}

In [26]:
index_to_char

{2: '^',
 3: 'о',
 4: 'а',
 5: 'и',
 6: 'е',
 7: 'в',
 8: 'н',
 9: 'р',
 10: 'с',
 11: 'т',
 12: 'л',
 13: 'м',
 14: 'п',
 15: 'к',
 16: 'у',
 17: 'я',
 18: 'ш',
 19: 'д',
 20: 'ы',
 21: 'з',
 22: 'й',
 23: 'ь',
 24: 'ю',
 25: 'г',
 26: 'б',
 27: 'ч',
 28: 'щ',
 29: 'х',
 30: 'ж',
 31: 'ц',
 32: 'ф',
 33: 'э',
 34: 'ъ',
 35: 'ё'}

#Векторизация

## Преобразуем слова в коды

Функция для преобразования списка букв в список кодов

In [27]:
def text_to_sequence(word, char_to_index):
    seq = []
    for char in word:
        index = char_to_index.get(char, 1) # 1 означает неизвестную букву
        # Неизвестные буквы не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

In [28]:
def stress_pos(word):
  i = 0
  for char in word:
    i += 1
    if char == '^':
      return i

In [29]:
word = data['Токены ударений'][15]

In [30]:
word

['а', 'а', 'р', 'о', '^', 'н', 'о', 'в', 'ц', 'а', 'м', 'и']

In [31]:
seq = stress_pos(word)

In [32]:
seq

5

In [33]:
word = data['Токены слов'][0]

In [34]:
word

['а', 'а', 'к']

In [35]:
seq = text_to_sequence(word, char_to_index)

In [36]:
seq

[4, 4, 15]

In [37]:
index_to_char[3]

'о'

In [38]:
seq = [char_to_index.get(char, 1) for char in word]

In [39]:
seq

[4, 4, 15]

Преобразуем все тексты в последовательность кодов слов

In [40]:
data['Номер ударений'] = data.apply(lambda row: stress_pos(row['Токены ударений']), axis=1)

In [41]:
data

,Ударения,Слова,Токены ударений,Токены слов,Номер ударений
0,аа^к,аак,"[а, а, ^, к]","[а, а, к]",3
1,аа^ка,аака,"[а, а, ^, к, а]","[а, а, к, а]",3
2,аа^ке,ааке,"[а, а, ^, к, е]","[а, а, к, е]",3
3,аа^ки,ааки,"[а, а, ^, к, и]","[а, а, к, и]",3
4,аа^ков,ааков,"[а, а, ^, к, о, в]","[а, а, к, о, в]",3
...,...,...,...,...,...
588485,ёфика^торе,ёфикаторе,"[ё, ф, и, к, а, ^, т, о, р, е]","[ё, ф, и, к, а, т, о, р, е]",6
588486,ёфика^ции,ёфикации,"[ё, ф, и, к, а, ^, ц, и, и]","[ё, ф, и, к, а, ц, и, и]",6
588487,ёфика^цию,ёфикацию,"[ё, ф, и, к, а, ^, ц, и, ю]","[ё, ф, и, к, а, ц, и, ю]",6
588488,ёфика^ция,ёфикация,"[ё, ф, и, к, а, ^, ц, и, я]","[ё, ф, и, к, а, ц, и, я]",6


In [42]:
data['Ударения'][2]

'аа^ке'

In [43]:
data['Токены ударений'][2]

['а', 'а', '^', 'к', 'е']

In [44]:
data['Номер ударений'][2]

3

In [45]:
data['Коды слов'] = data.apply(lambda row: text_to_sequence(row['Токены слов'], char_to_index), axis=1)

In [46]:
data

,Ударения,Слова,Токены ударений,Токены слов,Номер ударений,Коды слов
0,аа^к,аак,"[а, а, ^, к]","[а, а, к]",3,"[4, 4, 15]"
1,аа^ка,аака,"[а, а, ^, к, а]","[а, а, к, а]",3,"[4, 4, 15, 4]"
2,аа^ке,ааке,"[а, а, ^, к, е]","[а, а, к, е]",3,"[4, 4, 15, 6]"
3,аа^ки,ааки,"[а, а, ^, к, и]","[а, а, к, и]",3,"[4, 4, 15, 5]"
4,аа^ков,ааков,"[а, а, ^, к, о, в]","[а, а, к, о, в]",3,"[4, 4, 15, 3, 7]"
...,...,...,...,...,...,...
588485,ёфика^торе,ёфикаторе,"[ё, ф, и, к, а, ^, т, о, р, е]","[ё, ф, и, к, а, т, о, р, е]",6,"[35, 32, 5, 15, 4, 11, 3, 9, 6]"
588486,ёфика^ции,ёфикации,"[ё, ф, и, к, а, ^, ц, и, и]","[ё, ф, и, к, а, ц, и, и]",6,"[35, 32, 5, 15, 4, 31, 5, 5]"
588487,ёфика^цию,ёфикацию,"[ё, ф, и, к, а, ^, ц, и, ю]","[ё, ф, и, к, а, ц, и, ю]",6,"[35, 32, 5, 15, 4, 31, 5, 24]"
588488,ёфика^ция,ёфикация,"[ё, ф, и, к, а, ^, ц, и, я]","[ё, ф, и, к, а, ц, и, я]",6,"[35, 32, 5, 15, 4, 31, 5, 17]"


In [47]:
data['Слова'][2]

'ааке'

In [48]:
data['Токены слов'][2]

['а', 'а', 'к', 'е']

##Находим максимальную длину слов и составляем векторы

In [49]:
max = 0
indx = 0
i = 0
for row in data['Коды слов']:
  i+=1
  if len(row) > max:
    indx = i
    max = len(row)

In [50]:
max

36

In [51]:
indx

543271

In [54]:
def vectorize(seq,max):
  lst = seq.copy()
  for i in range(len(seq),max):
    lst.append(0)
  return np.array(lst)

In [55]:
lll = vectorize(data['Коды слов'][0],max)
lll

array([ 4,  4, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [56]:
data['Векторы слов'] = data.apply(lambda row: vectorize(row['Коды слов'],max), axis=1)

In [57]:
data

,Ударения,Слова,Токены ударений,Токены слов,Номер ударений,Коды слов,Векторы слов
0,аа^к,аак,"[а, а, ^, к]","[а, а, к]",3,"[4, 4, 15]","[4, 4, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,аа^ка,аака,"[а, а, ^, к, а]","[а, а, к, а]",3,"[4, 4, 15, 4]","[4, 4, 15, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,аа^ке,ааке,"[а, а, ^, к, е]","[а, а, к, е]",3,"[4, 4, 15, 6]","[4, 4, 15, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,аа^ки,ааки,"[а, а, ^, к, и]","[а, а, к, и]",3,"[4, 4, 15, 5]","[4, 4, 15, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,аа^ков,ааков,"[а, а, ^, к, о, в]","[а, а, к, о, в]",3,"[4, 4, 15, 3, 7]","[4, 4, 15, 3, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...
588485,ёфика^торе,ёфикаторе,"[ё, ф, и, к, а, ^, т, о, р, е]","[ё, ф, и, к, а, т, о, р, е]",6,"[35, 32, 5, 15, 4, 11, 3, 9, 6]","[35, 32, 5, 15, 4, 11, 3, 9, 6, 0, 0, 0, 0, 0,..."
588486,ёфика^ции,ёфикации,"[ё, ф, и, к, а, ^, ц, и, и]","[ё, ф, и, к, а, ц, и, и]",6,"[35, 32, 5, 15, 4, 31, 5, 5]","[35, 32, 5, 15, 4, 31, 5, 5, 0, 0, 0, 0, 0, 0,..."
588487,ёфика^цию,ёфикацию,"[ё, ф, и, к, а, ^, ц, и, ю]","[ё, ф, и, к, а, ц, и, ю]",6,"[35, 32, 5, 15, 4, 31, 5, 24]","[35, 32, 5, 15, 4, 31, 5, 24, 0, 0, 0, 0, 0, 0..."
588488,ёфика^ция,ёфикация,"[ё, ф, и, к, а, ^, ц, и, я]","[ё, ф, и, к, а, ц, и, я]",6,"[35, 32, 5, 15, 4, 31, 5, 17]","[35, 32, 5, 15, 4, 31, 5, 17, 0, 0, 0, 0, 0, 0..."


#Обучение модели

##Подготовка обучающей и тестовой выборки

In [58]:
new_arr = np.stack( np.array(data['Векторы слов']), axis=0 )

In [59]:
new_arr

array([[ 4,  4, 15, ...,  0,  0,  0],
       [ 4,  4, 15, ...,  0,  0,  0],
       [ 4,  4, 15, ...,  0,  0,  0],
       ...,
       [35, 32,  5, ...,  0,  0,  0],
       [35, 32,  5, ...,  0,  0,  0],
       [35, 32,  5, ...,  0,  0,  0]])

In [60]:
from sklearn.model_selection import train_test_split

y = data['Номер ударений'].copy()
X = new_arr

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=rand_state)

In [61]:
from sklearn.metrics import accuracy_score

In [62]:
y_train

229103     6
33128      6
154571     5
384278    10
39541      4
          ..
36482      7
564465     9
412925     6
255366     9
472907     5
Name: Номер ударений, Length: 411943, dtype: int64

In [63]:
X_train

array([[13,  3,  9, ...,  0,  0,  0],
       [26,  9,  6, ...,  0,  0,  0],
       [21,  5, 13, ...,  0,  0,  0],
       ...,
       [14,  9,  3, ...,  0,  0,  0],
       [ 8,  6, 19, ...,  0,  0,  0],
       [10,  4,  9, ...,  0,  0,  0]])

## Модель DecisionTreeClassifier

In [64]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [65]:
y_pred2 = dt.predict(X_test)

In [66]:
acc2 = accuracy_score(y_test,y_pred2)

In [67]:
acc2

0.8351657065823832

#Использование модели для прогнозирования

## Подготовка данных

###Если нужно прогнозирование для закрытого набора данных

In [ ]:
#test_private = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_D/private_test_stresses.txt', header=None, names=['Слова'])

In [ ]:
#Если загружать данные локально:
# test_private = pd.read_csv('private_test_stresses.txt', header=None, names=['Слова'])

In [ ]:
#test = test_private

###Если нужно прогнозировать для открытого датасета

In [71]:
test['Токены слов'] = test.apply(lambda row: preprocess(row['Слова']), axis=1)

In [72]:
test

,Слова,Токены слов
0,аакам,"[а, а, к, а, м]"
1,ааками,"[а, а, к, а, м, и]"
2,ааленец,"[а, а, л, е, н, е, ц]"
3,аама,"[а, а, м, а]"
4,аамами,"[а, а, м, а, м, и]"
...,...,...
294248,ёмиури,"[ё, м, и, у, р, и]"
294249,ёфикаторами,"[ё, ф, и, к, а, т, о, р, а, м, и]"
294250,ёфикатору,"[ё, ф, и, к, а, т, о, р, у]"
294251,ёфикациями,"[ё, ф, и, к, а, ц, и, я, м, и]"


In [73]:
test['Коды слов'] = test.apply(lambda row: text_to_sequence(row['Токены слов'], char_to_index), axis=1)

In [74]:
test

,Слова,Токены слов,Коды слов
0,аакам,"[а, а, к, а, м]","[4, 4, 15, 4, 13]"
1,ааками,"[а, а, к, а, м, и]","[4, 4, 15, 4, 13, 5]"
2,ааленец,"[а, а, л, е, н, е, ц]","[4, 4, 12, 6, 8, 6, 31]"
3,аама,"[а, а, м, а]","[4, 4, 13, 4]"
4,аамами,"[а, а, м, а, м, и]","[4, 4, 13, 4, 13, 5]"
...,...,...,...
294248,ёмиури,"[ё, м, и, у, р, и]","[35, 13, 5, 16, 9, 5]"
294249,ёфикаторами,"[ё, ф, и, к, а, т, о, р, а, м, и]","[35, 32, 5, 15, 4, 11, 3, 9, 4, 13, 5]"
294250,ёфикатору,"[ё, ф, и, к, а, т, о, р, у]","[35, 32, 5, 15, 4, 11, 3, 9, 16]"
294251,ёфикациями,"[ё, ф, и, к, а, ц, и, я, м, и]","[35, 32, 5, 15, 4, 31, 5, 17, 13, 5]"


In [75]:
test['Векторы слов'] = test.apply(lambda row: vectorize(row['Коды слов'],max), axis=1)

In [76]:
test

,Слова,Токены слов,Коды слов,Векторы слов
0,аакам,"[а, а, к, а, м]","[4, 4, 15, 4, 13]","[4, 4, 15, 4, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,ааками,"[а, а, к, а, м, и]","[4, 4, 15, 4, 13, 5]","[4, 4, 15, 4, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,ааленец,"[а, а, л, е, н, е, ц]","[4, 4, 12, 6, 8, 6, 31]","[4, 4, 12, 6, 8, 6, 31, 0, 0, 0, 0, 0, 0, 0, 0..."
3,аама,"[а, а, м, а]","[4, 4, 13, 4]","[4, 4, 13, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,аамами,"[а, а, м, а, м, и]","[4, 4, 13, 4, 13, 5]","[4, 4, 13, 4, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...,...
294248,ёмиури,"[ё, м, и, у, р, и]","[35, 13, 5, 16, 9, 5]","[35, 13, 5, 16, 9, 5, 0, 0, 0, 0, 0, 0, 0, 0, ..."
294249,ёфикаторами,"[ё, ф, и, к, а, т, о, р, а, м, и]","[35, 32, 5, 15, 4, 11, 3, 9, 4, 13, 5]","[35, 32, 5, 15, 4, 11, 3, 9, 4, 13, 5, 0, 0, 0..."
294250,ёфикатору,"[ё, ф, и, к, а, т, о, р, у]","[35, 32, 5, 15, 4, 11, 3, 9, 16]","[35, 32, 5, 15, 4, 11, 3, 9, 16, 0, 0, 0, 0, 0..."
294251,ёфикациями,"[ё, ф, и, к, а, ц, и, я, м, и]","[35, 32, 5, 15, 4, 31, 5, 17, 13, 5]","[35, 32, 5, 15, 4, 31, 5, 17, 13, 5, 0, 0, 0, ..."


In [77]:
new_arr = np.stack(np.array(test['Векторы слов']), axis=0 )

In [78]:
new_arr

array([[ 4,  4, 15, ...,  0,  0,  0],
       [ 4,  4, 15, ...,  0,  0,  0],
       [ 4,  4, 12, ...,  0,  0,  0],
       ...,
       [35, 32,  5, ...,  0,  0,  0],
       [35, 32,  5, ...,  0,  0,  0],
       [35, 32,  5, ...,  0,  0,  0]])

## Предсказание моделью DecisionTreeClassifier

In [79]:
X_testDF = new_arr

In [80]:
y_pred = dt.predict(X_testDF)
y_pred

array([3, 3, 5, ..., 6, 6, 6])

In [81]:
y_pred[3]

3

## Выполняем расстановку ударений по предсказаниям

In [82]:
test['Предсказания'] = y_pred.tolist()

In [83]:
def stress_put(word,index):
  i = 0
  for ch in word:
    i +=1
    if index>=len(word):
      word = word + '^'
      return word
    elif i==index:
      word = word[:index-1] + '^' + word[index-1:]
  return word

In [84]:
test['Ударения'] = test.apply(lambda row: stress_put(row['Слова'],row['Предсказания']), axis=1)

In [85]:
test

,Слова,Токены слов,Коды слов,Векторы слов,Предсказания,Ударения
0,аакам,"[а, а, к, а, м]","[4, 4, 15, 4, 13]","[4, 4, 15, 4, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3,аа^кам
1,ааками,"[а, а, к, а, м, и]","[4, 4, 15, 4, 13, 5]","[4, 4, 15, 4, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3,аа^ками
2,ааленец,"[а, а, л, е, н, е, ц]","[4, 4, 12, 6, 8, 6, 31]","[4, 4, 12, 6, 8, 6, 31, 0, 0, 0, 0, 0, 0, 0, 0...",5,аале^нец
3,аама,"[а, а, м, а]","[4, 4, 13, 4]","[4, 4, 13, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3,аа^ма
4,аамами,"[а, а, м, а, м, и]","[4, 4, 13, 4, 13, 5]","[4, 4, 13, 4, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3,аа^мами
...,...,...,...,...,...,...
294248,ёмиури,"[ё, м, и, у, р, и]","[35, 13, 5, 16, 9, 5]","[35, 13, 5, 16, 9, 5, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,ёми^ури
294249,ёфикаторами,"[ё, ф, и, к, а, т, о, р, а, м, и]","[35, 32, 5, 15, 4, 11, 3, 9, 4, 13, 5]","[35, 32, 5, 15, 4, 11, 3, 9, 4, 13, 5, 0, 0, 0...",6,ёфика^торами
294250,ёфикатору,"[ё, ф, и, к, а, т, о, р, у]","[35, 32, 5, 15, 4, 11, 3, 9, 16]","[35, 32, 5, 15, 4, 11, 3, 9, 16, 0, 0, 0, 0, 0...",6,ёфика^тору
294251,ёфикациями,"[ё, ф, и, к, а, ц, и, я, м, и]","[35, 32, 5, 15, 4, 31, 5, 17, 13, 5]","[35, 32, 5, 15, 4, 31, 5, 17, 13, 5, 0, 0, 0, ...",6,ёфика^циями


## Сохраняем расставленные ударения в файл

In [86]:
new_pred = test['Ударения'].copy()
new_pred

0               аа^кам
1              аа^ками
2             аале^нец
3                аа^ма
4              аа^мами
              ...     
294248         ёми^ури
294249    ёфика^торами
294250      ёфика^тору
294251     ёфика^циями
294252      ёфика^циях
Name: Ударения, Length: 294253, dtype: object

In [ ]:
# Если используется гугл диск
# new_pred.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_D/pred.txt',index=False, header=False)

In [87]:
# Если сохранять локально
new_pred.to_csv('public_pred.txt',index=False, header=False)

### Для закрытого датасета

In [ ]:
# Если используется гугл диск
# new_pred.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_D/pred_private.txt',index=False, header=False)

In [ ]:
# Если сохранять локально
new_pred.to_csv('pred_private.txt',index=False, header=False)